# Scraping a WIKI pages for make a DataSet

##### Import libraries

In [ ]:
from bs4 import BeautifulSoup as bs # for scrape
import requests

In [ ]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ',strip=True).replace('\xa0', ' ') for li in row_data.find_all('li')]
    else:
        return row_data.get_text(' ',strip=True).replace('\xa0', ' ')

def get_info_box(url):
    r = requests.get(url)

    soup = bs(r.content)
    info_box = soup.find(class_='infobox vevent')
    info_rows = info_box.find_all('tr')

    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(' ',strip=True)
        elif index == 1:
            continue
        else:    
            content_key = row.find('th').get_text(' ',strip=True)
            content_value = get_content_value(row.find('td'))
            movie_info[content_key] = content_value
    
    return movie_info


In [11]:
r = requests.get('https://en.wikipedia.org/wiki/1950_in_film')
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = 'https://en.wikipedia.org/'

movie_info_list = []
for index, movie in enumerate(movies):
    relative_path = movie['href']
    full_path = base_path+relative_path
    title = movie['title']
    
    movie_info_list.append(get_info_box(full_path))

TypeError: list indices must be integers or slices, not str